In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import classification_report, precision_score, recall_score, confusion_matrix, roc_curve, auc, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix
# display confusion matrix:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
#from tune_sklearn import TuneSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
import itertools
from sklearn.preprocessing import LabelBinarizer
import imblearn
import lightgbm as lgb
targs = ['target_question','target_discarded','target_subsection','target_section']


In [2]:
%%time
file_path = 'GT_IPI.parquet'
df = pd.read_parquet(file_path, engine='pyarrow')

CPU times: total: 16.2 s
Wall time: 5.06 s


In [3]:
indices_df = pd.read_csv('unique_indexes.csv')
indices = indices_df['unique_index'].tolist()
filtered_df = df[df.index.isin(indices)]

# If you want to reset the index after filtering (optional)
filtered_df.reset_index(drop=True, inplace=True)
df = filtered_df

In [4]:
bool_column = df.apply({0,1}.issuperset)
bool_column_list = bool_column[bool_column == True].index.to_list()
style_counts_by_target = df.groupby('target')['style'].value_counts()
# too many style value in the column style
style_counts_by_target = df.groupby('target')['style'].value_counts()

# Convert the result to a DataFrame
style_counts_by_target_df = style_counts_by_target.to_frame(name='Count').reset_index()

# Initialize an empty list to hold the top 10 terms for each target
top_15_per_target = []

# Calculate the percentage representation and get the top 15 terms for each 'target'
for target in style_counts_by_target_df['target'].unique():
    # Filter the DataFrame for the current target
    target_df = style_counts_by_target_df[style_counts_by_target_df['target'] == target]
    
    # Calculate the total number of terms for the current target
    total_terms = target_df['Count'].sum()
    
    # Calculate the percentage representation of each term
    target_df['Percentage'] = (target_df['Count'] / total_terms) * 100
    
    # Get the top 15 terms
    top_15 = target_df.head(15)
    
    # Append the result to the list
    top_15_per_target.append(top_15)

# Concatenate all top 10 DataFrames into a single DataFrame
combined_top_15 = pd.concat(top_15_per_target)

# Display the unique 'style' values in the combined DataFrame
unique_styles = combined_top_15['style'].unique()
## manually remove title from the list as it is duplicated
#unique_styles = np.delete(unique_styles, np.where(unique_styles == 'title'))
print("Unique 'style' values in the top 15 per target:")
print(unique_styles)
# Transform the 'style' column to 'None' if the current value isn't found in unique_styles
df['style'] = df['style'].apply(lambda x: x if x in unique_styles else 'None')
# Keep a copy of the original 'target' column
original_target = df['target'].copy()

# Apply one-hot encoding to the specified columns
#df_encoded = pd.get_dummies(df, columns=['target', 'begins_with', 'style'], prefix=['target', 'begins_with', 'style'])
df_encoded = pd.get_dummies(df,drop_first = True
                            , columns=['begins_with','para_foll_depth_ind','para_foll_size_ind', 'para_prec_depth_ind', 'para_prec_size_ind', 'style'] + bool_column_list\
                            , prefix=['begins_with','para_foll_depth_ind','para_foll_size_ind', 'para_prec_depth_ind', 'para_prec_size_ind', 'style'] + bool_column_list)
df_encoded = pd.get_dummies(df_encoded
                            , columns=['target']\
                            , prefix=['target'] )

# Add the original 'target' column back to the encoded DataFrame
df_encoded['target'] = original_target



C:\Users\wyatt\AppData\Local\Temp\ipykernel_19632\793476661.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['Percentage'] = (target_df['Count'] / total_terms) * 100
C:\Users\wyatt\AppData\Local\Temp\ipykernel_19632\793476661.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_df['Percentage'] = (target_df['Count'] / total_terms) * 100
C:\Users\wyatt\AppData\Local\Temp\ipykernel_19632\793476661.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

Unique 'style' values in the top 15 per target:
['default' 'listparagraph' 'bodytext' 'tableparagraph' 'normal' 'heading2'
 'toc2' 'toc1' 'heading1' 'nospacing' 'heading3' 'normalweb' 'tabletext'
 'toc3' 'listnumber' 'question' 'dvquestion' 'paragraphedeliste'
 'esisquestion' 'title' 'heading4' 'heading10' 'paragraph' 'titre1'
 'dvsubcategory' 'rfpheading3' 'himheading4bodybold' 'heading20']


In [5]:
customers_with_less_than2_csspk = [
    x for x in df_encoded['customer_pk'].unique() 
    if len(df_encoded[df_encoded['customer_pk'] == x]['css_pk'].unique()) < 2
]

filtered_df = df_encoded[~df_encoded['customer_pk'].isin(customers_with_less_than2_csspk)]

In [6]:
#array(['question', 'discarded', 'subsection', 'section'],
exclude = ['lang_num_sents','css_pk','customer_pk','html_pk','id']
#top_15_columns
top_15_columns = []
columns_to_exclude = exclude + top_15_columns

In [8]:
def train_test(target,sampling_tech='none'):
    X_train = train_df.drop(columns=columns_to_exclude + ['target'] + targs)
    y_train = train_df[f'target_{target}']
    X_test = test_df.drop(columns=columns_to_exclude + ['target'] + targs)
    y_test = test_df[f'target_{target}']
    if sampling_tech == 'under':
        rus = imblearn.under_sampling.RandomUnderSampler(random_state = 420)
    if sampling_tech == 'over':
        rus = imblearn.over_sampling.RandomOverSampler(random_state = 420)
    if sampling_tech == 'over' or sampling_tech == 'under':
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
        X_train = X_train_resampled
        y_train = y_train_resampled
    model = lgb.LGBMClassifier(objective='binary', metric='auc')
    model.fit(X_train, y_train)
    predicts = model.predict_proba(X_test)
    return predicts
def multi_pred_stats(predicts_list):
    true_question = [subarray[1] for subarray in predicts_list[0]]
    true_discarded = [subarray[1] for subarray in predicts_list[1]]
    true_subsection = [subarray[1] for subarray in predicts_list[2]]
    true_section = [subarray[1] for subarray in predicts_list[3]]

    stacked_arrays = np.vstack((true_question, true_discarded, true_section, true_subsection))
    label_names = ['question','discarded','section','subsection']
    max_indices = np.argmax(stacked_arrays, axis=0)
    result = [label_names[i] for i in max_indices]

# Initialize a dictionary to hold counts
    accuracy_dict = {key: {'correct': 0, 'total': 0} for key in test_df['target'].unique()}

# Iterate over the target and result lists to count occurrences and correct predictions
    for t, p in zip(test_df['target'], result):
        accuracy_dict[t]['total'] += 1
        if t == p:
            accuracy_dict[t]['correct'] += 1
    
    # Calculate accuracy for each string
    for key in accuracy_dict:
        correct = accuracy_dict[key]['correct']
        total = accuracy_dict[key]['total']
        accuracy = correct / total if total != 0 else 0
        accuracy_dict[key]['accuracy'] = accuracy
    
    # Convert the result to a DataFrame for better visualization
    accuracy_df = pd.DataFrame(accuracy_dict).transpose()
    return accuracy_df

In [9]:
# List of parameter combinations
combinations = [
    ('question', 'none', 'discarded', 'none', 'subsection', 'none', 'section', 'none'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'none', 'section', 'over'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'none', 'section', 'under'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'over', 'section', 'none'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'over', 'section', 'over'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'over', 'section', 'under'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'under', 'section', 'none'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'under', 'section', 'over'),
    ('question', 'none', 'discarded', 'none', 'subsection', 'under', 'section', 'under'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'none', 'section', 'none'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'none', 'section', 'over'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'none', 'section', 'under'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'over', 'section', 'none'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'over', 'section', 'over'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'over', 'section', 'under'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'under', 'section', 'none'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'under', 'section', 'over'),
    ('question', 'none', 'discarded', 'over', 'subsection', 'under', 'section', 'under'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'none', 'section', 'none'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'none', 'section', 'over'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'none', 'section', 'under'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'over', 'section', 'none'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'over', 'section', 'over'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'over', 'section', 'under'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'under', 'section', 'none'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'under', 'section', 'over'),
    ('question', 'none', 'discarded', 'under', 'subsection', 'under', 'section', 'under'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'none', 'section', 'none'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'none', 'section', 'over'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'none', 'section', 'under'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'over', 'section', 'none'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'over', 'section', 'over'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'over', 'section', 'under'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'under', 'section', 'none'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'under', 'section', 'over'),
    ('question', 'over', 'discarded', 'none', 'subsection', 'under', 'section', 'under'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'none', 'section', 'none'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'none', 'section', 'over'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'none', 'section', 'under'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'over', 'section', 'none'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'over', 'section', 'over'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'over', 'section', 'under'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'under', 'section', 'none'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'under', 'section', 'over'),
    ('question', 'over', 'discarded', 'over', 'subsection', 'under', 'section', 'under'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'none', 'section', 'none'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'none', 'section', 'over'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'none', 'section', 'under'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'over', 'section', 'none'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'over', 'section', 'over'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'over', 'section', 'under'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'under', 'section', 'none'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'under', 'section', 'over'),
    ('question', 'over', 'discarded', 'under', 'subsection', 'under', 'section', 'under'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'none', 'section', 'none'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'none', 'section', 'over'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'none', 'section', 'under'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'over', 'section', 'none'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'over', 'section', 'over'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'over', 'section', 'under'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'under', 'section', 'none'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'under', 'section', 'over'),
    ('question', 'under', 'discarded', 'none', 'subsection', 'under', 'section', 'under'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'none', 'section', 'none'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'none', 'section', 'over'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'none', 'section', 'under'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'over', 'section', 'none'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'over', 'section', 'over'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'over', 'section', 'under'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'under', 'section', 'none'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'under', 'section', 'over'),
    ('question', 'under', 'discarded', 'over', 'subsection', 'under', 'section', 'under'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'none', 'section', 'none'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'none', 'section', 'over'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'none', 'section', 'under'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'over', 'section', 'none'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'over', 'section', 'over'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'over', 'section', 'under'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'under', 'section', 'none'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'under', 'section', 'over'),
    ('question', 'under', 'discarded', 'under', 'subsection', 'under', 'section', 'under')
]
#accuracy_dictionaries = []
### Iterate over the combinations and call the function
#for combo in combinations[55::]:
#    accuracy_dictionaries.append(multi_pred_stats([
#        train_test(combo[0], sampling_tech=combo[1]),
#        train_test(combo[2], sampling_tech=combo[3]),
#        train_test(combo[4], sampling_tech=combo[5]),
#        train_test(combo[6], sampling_tech=combo[7])    ]))



In [13]:
('question', 'under', 'discarded', 'under', 'subsection', 'none', 'section', 'under')
             correct     total  accuracy
section      10700.0   13070.0  0.818669
question    255421.0  317052.0  0.805612
discarded   251413.0  336199.0  0.747810
subsection     232.0    9752.0  0.023790
('question', 'under', 'discarded', 'under', 'subsection', 'under', 'section', 'none')
             correct     total  accuracy
section        196.0   13070.0  0.014996
question    252339.0  317052.0  0.795892
discarded   249302.0  336199.0  0.741531
subsection    6811.0    9752.0  0.698421
('question', 'over', 'discarded', 'over', 'subsection', 'none', 'section', 'under')
             correct     total  accuracy
section      10693.0   13070.0  0.818133
question    255346.0  317052.0  0.805376
discarded   251524.0  336199.0  0.748140
subsection     232.0    9752.0  0.023790
('question', 'none', 'discarded', 'under', 'subsection', 'none', 'section', 'over')
             correct     total  accuracy
section      10718.0   13070.0  0.820046
question    252005.0  317052.0  0.794838
discarded   254889.0  336199.0  0.758149
subsection     235.0    9752.0  0.024098


IndentationError: unexpected indent (4072426336.py, line 2)

In [10]:
from sklearn.model_selection import GroupShuffleSplit

# Lists to store the different train and test dataframes
r_state = [1, 2, 3, 4, 5]
train_df_dict = dict()
test_df_dict = dict()

# Generate 5 different train/test splits
for r in r_state:
    gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=10*r)
    train_list = []
    test_list = []

    # Perform train-test split within each group
    for i in filtered_df['customer_pk'].unique():
        X = filtered_df.loc[filtered_df['customer_pk'] == i]
        train_index, test_index = next(gss.split(X, groups=X['css_pk']))
        
        # Append the subset data to the respective list
        train_list.append(X.iloc[train_index])
        test_list.append(X.iloc[test_index])

    # Concatenate the lists to create final train and test DataFrames
    train_df_dict[r] = pd.concat(train_list, ignore_index=True)
    test_df_dict[r] = pd.concat(test_list, ignore_index=True)

# Check the length of one of the train and test sets
print(f"Train DataFrame length for r=1: {len(train_df_dict[1])}")
print(f"Test DataFrame length for r=1: {len(test_df_dict[1])}")

Train DataFrame length for r=1: 2438929
Test DataFrame length for r=1: 680281


In [11]:
from sklearn.metrics import log_loss
from sklearn.metrics import average_precision_score
import pickle
import lightgbm as lgb
from bayes_opt import BayesianOptimization
t = 'question'
# Define the expanded parameter space
param_space_expanded = {
    'num_leaves': (1, 100),
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.8),
    'n_estimators': (1, 1000),
    'min_child_samples': (10, 100),
    'bagging_fraction': (0.01, 1.0),
    'feature_fraction': (0.01, 1.0),
    'lambda_l1': (0.0, 10.0),
    'lambda_l2': (0.0, 10.0),
    'min_split_gain': (0.0, 1.0),
    'min_child_weight':(0.01,20)
}

# Define the objective function
def lgb_eval(num_leaves, max_depth, learning_rate, n_estimators, min_child_samples,
             bagging_fraction, feature_fraction, lambda_l1, lambda_l2, min_split_gain,  min_child_weight):
    params = {
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'min_child_samples': int(min_child_samples),
        'bagging_fraction': bagging_fraction,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_split_gain': min_split_gain,
        'min_child_weight':min_child_weight,
        'objective': 'binary',
        'metric': 'average_precision',
        'boosting_type': 'gbdt',
        'n_jobs':-1,
        'device_type':'gpu'
    }
    aps_scores = []
    for r in r_state:
        # Data preparation
        train_df = train_df_dict[r]
        test_df = test_df_dict[r]
        X_train = train_df.drop(columns=columns_to_exclude + ['target'] + targs)
        y_train = train_df[f'target_{t}']
        X_test = test_df.drop(columns=columns_to_exclude + ['target'] + targs)
        y_test = test_df[f'target_{t}']
        if t=='discarded':
            rus = imblearn.under_sampling.RandomUnderSampler()
            X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
            X_train = X_train_resampled
            y_train = y_train_resampled
        if t=='section':
            rus = imblearn.over_sampling.RandomOverSampler()
            X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
            X_train = X_train_resampled
            y_train = y_train_resampled
        print(params)
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        aps = average_precision_score(y_test, y_pred)
        aps_scores.append(aps)
        print(f'AVERAGE PRECISION SCORE: {aps}')
    return np.mean(aps_scores)

# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=lgb_eval,
    pbounds=param_space_expanded,
    random_state=42,
    verbose=1
)

# Perform optimization
optimizer.maximize(init_points=2, n_iter=5)

# Best parameters
best_params = optimizer.max['params']
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_samples'] = int(best_params['min_child_samples'])

print("Best Parameters:", best_params)

# Train the final model with the best parameters
#final_model = lgb.LGBMClassifier(**best_params)
#final_model.fit(X_train, y_train)
with open('best_params1.pkl', 'wb') as pickle_file:
    pickle.dump(best_params, pickle_file)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_ch... | min_sp... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
{'num_leaves': 3, 'max_depth': 5, 'learning_rate': 0.13325472594952487, 'n_estimators': 708, 'min_child_samples': 15, 'bagging_fraction': 0.3807947176588889, 'feature_fraction': 0.951207163345817, 'lambda_l1': 7.319939418114051, 'lambda_l2': 5.986584841970366, 'min_split_gain': 0.6011150117432088, 'min_child_weight': 17.324861154040953, 'objective': 'binary', 'metric': 'average_precision', 'boosting_type': 'gbdt', 'n_jobs': -1, 'device_type': 'gpu'}
[LightGBM] [Warning] feature_fraction is set=0.951207163345817, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.951207163345817
[LightGBM] [Warning] lambda_l1 is set=7.319939418114051, reg_alpha=0.0 will be

In [ ]:
# Best parameters
best_params = optimizer.max['params']
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_samples'] = int(best_params['min_child_samples'])

print("Best Parameters:", best_params)

In [12]:
from sklearn.metrics import log_loss
from sklearn.metrics import average_precision_score
import pickle
import lightgbm as lgb
from bayes_opt import BayesianOptimization
# Define the expanded parameter space
param_space_expanded = {
    'num_leaves': (1, 100),
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.8),
    'n_estimators': (1, 1000),
    'min_child_samples': (10, 100),
    'bagging_fraction': (0.01, 1.0),
    'feature_fraction': (0.01, 1.0),
    'lambda_l1': (0.0, 10.0),
    'lambda_l2': (0.0, 10.0),
    'min_split_gain': (0.0, 1.0),
    'min_child_weight':(0.01,20)
}

# Define the objective function
def lgb_eval(num_leaves, max_depth, learning_rate, n_estimators, min_child_samples,
             bagging_fraction, feature_fraction, lambda_l1, lambda_l2, min_split_gain,  min_child_weight):
    params = {
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'min_child_samples': int(min_child_samples),
        'bagging_fraction': bagging_fraction,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_split_gain': min_split_gain,
        'min_child_weight':min_child_weight,
        'objective': 'binary',
        'metric': 'average_precision',
        'boosting_type': 'gbdt',
        'n_jobs':-1,
        'device_type':'gpu'
    }
    aps_scores = []
    for r in r_state:
        # Data preparation
        train_df = train_df_dict[r]
        test_df = test_df_dict[r]
        X_train = train_df.drop(columns=columns_to_exclude + ['target'] + targs)
        y_train = train_df[f'target_{t}']
        X_test = test_df.drop(columns=columns_to_exclude + ['target'] + targs)
        y_test = test_df[f'target_{t}']
        if t=='discarded':
            rus = imblearn.under_sampling.RandomUnderSampler()
            X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
            X_train = X_train_resampled
            y_train = y_train_resampled
        if t=='section':
            rus = imblearn.over_sampling.RandomOverSampler()
            X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
            X_train = X_train_resampled
            y_train = y_train_resampled
        print(params)
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        aps = average_precision_score(y_test, y_pred)
        aps_scores.append(aps)
        print(f'AVERAGE PRECISION SCORE: {aps_scores}')
    return np.mean(aps_scores)
t = 'discarded'

# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=lgb_eval,
    pbounds=param_space_expanded,
    random_state=42,
    verbose=2
)

# Perform optimization
optimizer.maximize(init_points=1, n_iter=10)

# Best parameters
best_params = optimizer.max['params']
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_samples'] = int(best_params['min_child_samples'])

print("Best Parameters:", best_params)

# Train the final model with the best parameters
#final_model2 = lgb.LGBMClassifier(**best_params)
#final_model2.fit(X_train, y_train)
with open('best_params2.pkl', 'wb') as pickle_file:
    pickle.dump(best_params, pickle_file)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_ch... | min_sp... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
{'num_leaves': 3, 'max_depth': 5, 'learning_rate': 0.13325472594952487, 'n_estimators': 708, 'min_child_samples': 15, 'bagging_fraction': 0.3807947176588889, 'feature_fraction': 0.951207163345817, 'lambda_l1': 7.319939418114051, 'lambda_l2': 5.986584841970366, 'min_split_gain': 0.6011150117432088, 'min_child_weight': 17.324861154040953, 'objective': 'binary', 'metric': 'average_precision', 'boosting_type': 'gbdt', 'n_jobs': -1, 'device_type': 'gpu'}
[LightGBM] [Warning] feature_fraction is set=0.951207163345817, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.951207163345817
[LightGBM] [Warning] lambda_l1 is set=7.319939418114051, reg_alpha=0.0 will be

In [13]:
t = 'section'
# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=lgb_eval,
    pbounds=param_space_expanded,
    random_state=42,
    verbose=2
)

# Perform optimization
optimizer.maximize(init_points=1, n_iter=10)

# Best parameters
best_params = optimizer.max['params']
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_samples'] = int(best_params['min_child_samples'])

print("Best Parameters:", best_params)

# Train the final model with the best parameters
#final_model3 = lgb.LGBMClassifier(**best_params)
#final_model3.fit(X_train, y_train)
with open('best_params3.pkl', 'wb') as pickle_file:
    pickle.dump(best_params, pickle_file)

|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_ch... | min_sp... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
{'num_leaves': 3, 'max_depth': 5, 'learning_rate': 0.13325472594952487, 'n_estimators': 708, 'min_child_samples': 15, 'bagging_fraction': 0.3807947176588889, 'feature_fraction': 0.951207163345817, 'lambda_l1': 7.319939418114051, 'lambda_l2': 5.986584841970366, 'min_split_gain': 0.6011150117432088, 'min_child_weight': 17.324861154040953, 'objective': 'binary', 'metric': 'average_precision', 'boosting_type': 'gbdt', 'n_jobs': -1, 'device_type': 'gpu'}
[LightGBM] [Warning] feature_fraction is set=0.951207163345817, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.951207163345817
[LightGBM] [Warning] lambda_l1 is set=7.319939418114051, reg_alpha=0.0 will be

In [15]:
from sklearn.metrics import log_loss
from sklearn.metrics import average_precision_score
import pickle
import lightgbm as lgb
from bayes_opt import BayesianOptimization
# Define the expanded parameter space
param_space_expanded = {
    'num_leaves': (1, 100),
    'max_depth': (3, 20),
    'learning_rate': (0.01, 0.8),
    'n_estimators': (1, 1000),
    'min_child_samples': (10, 100),
    'bagging_fraction': (0.01, 1.0),
    'feature_fraction': (0.01, 1.0),
    'lambda_l1': (0.0, 10.0),
    'lambda_l2': (0.0, 10.0),
    'min_split_gain': (0.0, 1.0),
    'min_child_weight':(0.01,20)
}

# Define the objective function
def lgb_eval(num_leaves, max_depth, learning_rate, n_estimators, min_child_samples,
             bagging_fraction, feature_fraction, lambda_l1, lambda_l2, min_split_gain,  min_child_weight):
    params = {
        'num_leaves': int(num_leaves),
        'max_depth': int(max_depth),
        'learning_rate': learning_rate,
        'n_estimators': int(n_estimators),
        'min_child_samples': int(min_child_samples),
        'bagging_fraction': bagging_fraction,
        'feature_fraction': feature_fraction,
        'lambda_l1': lambda_l1,
        'lambda_l2': lambda_l2,
        'min_split_gain': min_split_gain,
        'min_child_weight':min_child_weight,
        'objective': 'binary',
        'metric': 'average_precision',
        'boosting_type': 'gbdt',
        'n_jobs':-1,
        'device_type':'gpu'
    }
    aps_scores = []
    for r in r_state:
        # Data preparation
        train_df = train_df_dict[r]
        test_df = test_df_dict[r]
        X_train = train_df.drop(columns=columns_to_exclude + ['target'] + targs)
        y_train = train_df[f'target_{t}']
        X_test = test_df.drop(columns=columns_to_exclude + ['target'] + targs)
        y_test = test_df[f'target_{t}']
        if t=='discarded':
            rus = imblearn.under_sampling.RandomUnderSampler()
            X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
            X_train = X_train_resampled
            y_train = y_train_resampled
        if t=='section' or t=='subsection':
            rus = imblearn.over_sampling.RandomOverSampler()
            X_train_resampled, y_train_resampled = rus.fit_resample(X_train, y_train)
            X_train = X_train_resampled
            y_train = y_train_resampled
        print(params)
        model = lgb.LGBMClassifier(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict_proba(X_test)[:, 1]
        aps = average_precision_score(y_test, y_pred)
        aps_scores.append(aps)
        print(f'AVERAGE PRECISION SCORE: {aps_scores}')
    return np.mean(aps_scores)
t='subsection'
# Initialize Bayesian Optimization
optimizer = BayesianOptimization(
    f=lgb_eval,
    pbounds=param_space_expanded,
    random_state=42,
    verbose=2
)

# Perform optimization
optimizer.maximize(init_points=1, n_iter=10)

# Best parameters
best_params = optimizer.max['params']
best_params['num_leaves'] = int(best_params['num_leaves'])
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_samples'] = int(best_params['min_child_samples'])

print("Best Parameters:", best_params)
with open('best_params4.pkl', 'wb') as pickle_file:
    pickle.dump(best_params, pickle_file)
# Train the final model with the best parameters
#final_model4 = lgb.LGBMClassifier(**best_params)
#final_model4.fit(X_train, y_train)


|   iter    |  target   | baggin... | featur... | lambda_l1 | lambda_l2 | learni... | max_depth | min_ch... | min_ch... | min_sp... | n_esti... | num_le... |
-------------------------------------------------------------------------------------------------------------------------------------------------------------
{'num_leaves': 3, 'max_depth': 5, 'learning_rate': 0.13325472594952487, 'n_estimators': 708, 'min_child_samples': 15, 'bagging_fraction': 0.3807947176588889, 'feature_fraction': 0.951207163345817, 'lambda_l1': 7.319939418114051, 'lambda_l2': 5.986584841970366, 'min_split_gain': 0.6011150117432088, 'min_child_weight': 17.324861154040953, 'objective': 'binary', 'metric': 'average_precision', 'boosting_type': 'gbdt', 'n_jobs': -1, 'device_type': 'gpu'}
[LightGBM] [Warning] feature_fraction is set=0.951207163345817, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.951207163345817
[LightGBM] [Warning] lambda_l1 is set=7.319939418114051, reg_alpha=0.0 will be